In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tabulate import tabulate

In [ ]:
# load the database

db_df = pd.read_csv('/content/website_toycar.csv')
db_df

In [ ]:
#  separate data rows based on session id and match them to unique id

cases = [[87, "autistic"], [88, "autistic"], [89, "normal"], [94, "normal"], [95, "autistic"], [100, "autistic"],
         [102, "normal"], [106, "autistic"], [108, "NA"], [110, "autistic"], [112, "autistic"], [114, "autistic"],
         [116, "autistic"], [118, "autistic"], [120, "NA"], [121, "NA"], [124, "NA"]]

# create data list
# [id,status,dataframe]

db = []
session_id_list = db_df["session_id"].unique()
for case in cases:
  if case[0] in session_id_list:
    db.append([case[0], case[1], db_df.loc[db_df["session_id"] == case[0]] ])
  else:
    db.append([case[0], case[1], pd.DataFrame()])

In [ ]:
#  remove offset from all dataframes

for entry in db:
  if not entry[2].empty:
    temp = entry[2]
    entry[2] = entry[2] - [temp.iloc[0]["id"], temp.iloc[0]["time"], 0, 0, 0, temp.iloc[0]["encode1"], temp.iloc[0]["encode2"], 0]

# manualy correct outlier time data
db[4][2]["time"].iloc[522:] = db[4][2]["time"].iloc[522:] + 3415735

In [ ]:
# plot raw data

for entry in db:
  if not entry[2].empty:
    fig, axes = plt.subplots(nrows=1, ncols=2, figsize = (20,8))
    entry[2].plot(ax=axes[0], x = "time" , y = ["ac_x","ac_y","ac_z"], title = str(entry[0])+entry[1])
    entry[2].plot(ax=axes[1], x = "time" , y = ["encode1","encode2"], title = str(entry[0])+entry[1])
    plt.show()

In [ ]:
# wavelet filter functions for ACC filtering

import pywt
import warnings
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

warnings.filterwarnings("ignore")

def madev(d, axis=None):
    """ Mean absolute deviation of a signal """
    return np.mean(np.absolute(d - np.mean(d, axis)), axis)



def wavelet_denoising(x, wavelet='db4', level=1):
    coeff = pywt.wavedec(x, wavelet, mode="per")
    sigma = (1/0.6745) * madev(coeff[-level])
    uthresh = sigma * np.sqrt(2 * np.log(len(x)))
    coeff[1:] = (pywt.threshold(i, value=uthresh, mode='hard') for i in coeff[1:])
    return pywt.waverec(coeff, wavelet, mode='per')

In [ ]:
# smooth ACC data
# filter encoder data

filtered_data = []
# [id, stat, [time,Acx, Acy, Acz, enc1, enc2]]
for entry in db:
  if not entry[2].empty:
    temp = entry[2]
    acx = temp["ac_x"].to_numpy()
    acy = temp["ac_y"].to_numpy()
    acz = temp["ac_z"].to_numpy()

    time = temp["time"].to_numpy()

    enc1 = temp["encode1"].to_numpy()
    enc2 = temp["encode2"].to_numpy()

    enc1_dot = np.diff(enc1)
    enc2_dot = np.diff(enc2)

    j_inx_1 = np.argwhere(enc1_dot != 0)
    j_inx_2 = np.argwhere(enc2_dot != 0)

    fenc1 = np.zeros(len(enc1))
    fenc2 = np.zeros(len(enc2))

    for idx in j_inx_1:
      idx = int(idx)
      fenc1[idx:] += 1

    for idxx in j_inx_2:
      idxx = int(idxx)
      fenc2[idxx:] += 1

    acx_filtered = wavelet_denoising(acx, wavelet='sym16', level=1)
    acy_filtered = wavelet_denoising(acy, wavelet='sym16', level=1)
    acz_filtered = wavelet_denoising(acz, wavelet='sym16', level=1)

    filtered_data.append([entry[0], entry[1], [time,acx_filtered, acy_filtered, acz_filtered, fenc1, fenc2]])

In [ ]:
def windower(index_array, win_size = 4):
  windows_list = []
  if win_size % 2 != 0:
    raise Exception("window size must be even")
  for point in index_array:
    if point[0] - win_size/2 <= 0:
      start_point = 0
      end_point = point[0] + win_size/2
    elif point[0] + win_size/2 >= index_array[-1][0]:
      start_point = point[0] - win_size/2
      end_point = index_array[-1][0]
    else:
      start_point = point[0] - win_size/2
      end_point = point[0] + win_size/2

    windows_list.append([int(start_point), int(end_point)])
  return windows_list

In [ ]:
def zero_filter(input_np, error_per):
  M = np.max(input_np)
  up = error_per * M
  down = -1 * error_per * M

  output = input_np
  for i in range(len(output)):
    if output[i] <= up and output[i] >= down:
      output[i] = 0
    else:
      continue

  return output

In [ ]:
def mod_zero_filter(input_np, error_per):
  M = np.std(input_np)
  up = error_per * M
  down = -1 * error_per * M

  output = input_np
  for i in range(len(output)):
    if output[i] <= up and output[i] >= down:
      output[i] = 0
    else:
      continue

  return output

In [ ]:
def mapper(input, out_down = -100, out_up = 100):
  # out_up = 100
  # out_down = -100
  
  in_up = max(input)
  in_down = min(input)

  y = out_down + ((out_up - out_down)/(in_up - in_down)) * (input - in_down)

  return y

In [ ]:
def search_sequence_numpy(arr,seq):
    """ Find sequence in an array using NumPy only.

    Parameters
    ----------    
    arr    : input 1D array
    seq    : input 1D array

    Output
    ------    
    Output : 1D Array of indices in the input array that satisfy the 
    matching of input sequence in the input array.
    In case of no match, an empty list is returned.
    """

    # Store sizes of input array and sequence
    Na, Nseq = arr.size, seq.size

    # Range of sequence
    r_seq = np.arange(Nseq)

    # Create a 2D array of sliding indices across the entire length of input array.
    # Match up with the input sequence & get the matching starting indices.
    M = (arr[np.arange(Na-Nseq+1)[:,None] + r_seq] == seq).all(1)

    # Get the range of those indices as final output
    if M.any() >0:
        return np.where(np.convolve(M,np.ones((Nseq),dtype=int))>0)[0]
    else:
        return []         # No match found

In [ ]:
def event_cntr(diff, enc, win_size = 8, threshold = 0.02):

  diff[np.argwhere(diff != 0)] = 1
  rise_edge = search_sequence_numpy(diff, np.array([1,0]))
  fall_edge = search_sequence_numpy(diff, np.array([0,1]))

  total = np.concatenate((rise_edge[::2], fall_edge[::2]), axis=None)
  total = np.sort(total)

  stop_ratio = 0
  wheel_ratio = 0
  air_ratio = 0
  gnd_ratio = 0

  for i in range(len(total) - 1):
    if total[i+1] - total[i] >= win_size: # where acc dot is almost zero and without bouncing
      if (enc[total[i+1]] - enc[total[i]]) != 0 and (enc[total[i+1]] - enc[total[i]])/(total[i+1] - total[i]) > threshold: # the wheels are moving
        wheel_ratio += (total[i+1] - total[i])
      else: # the wheels not moving, total stop
        stop_ratio += (total[i+1] - total[i])
    else: # acc dot is not zero
      if (enc[total[i+1]] - enc[total[i]]) != 0  and (enc[total[i+1]] - enc[total[i]])/(total[i+1] - total[i]) > threshold: # the wheels are moving
        gnd_ratio += (total[i+1] - total[i])
      else: # the wheels not moving, in the air
        air_ratio += (total[i+1] - total[i])

  table = [stop_ratio, wheel_ratio, air_ratio, gnd_ratio]
  sum = stop_ratio + wheel_ratio + air_ratio + gnd_ratio

  return table/sum

In [ ]:
# plot filltered data

interval_window = 16
table = []

for temp in filtered_data:
  id = temp[0]
  stat = temp[1]
  data = temp[2]

  name = str(id)+stat

  time = data[0]

  accx = mapper(data[1], -9.8, 9.8)
  accy = mapper(data[2], -9.8, 9.8)
  accz = mapper(data[3], -9.8, 9.8)

  enc1 = data[4]
  enc2 = data[5]

  fig, axs = plt.subplots(nrows=1, ncols=2, figsize=(20,8))


  axs[0].plot(accx, label='Acx')
  axs[0].plot(accy, label='Acy')
  axs[0].plot(accz, label='Acz')
  axs[0].set_title('ACC')
  axs[0].legend()


  axs[1].plot(enc1, label='enc1')
  axs[1].plot(enc2, label='enc2')
  axs[1].set_title('encoders')
  axs[1].legend()

  fig.suptitle(name)

  plt.show()


In [ ]:
# event classification based on ACCs
# don't work properly

win_size = 16
threshold = 0.2
table = []

for temp in filtered_data:
  id = temp[0]
  stat = temp[1]
  data = temp[2]

  name = str(id)+stat

  time = data[0]

  accx = data[1]
  accy = data[2]
  accz = data[3]

  enc1 = data[4]
  enc2 = data[5]

  acc = np.sqrt(accx **2 + accy **2 + accz **2)
  # acc = wavelet_denoising(acc, wavelet='sym16', level=2)
  facc = zero_filter(np.diff(acc), 0.001)
  

  diff_acc = np.copy(facc)
  diff_acc[np.argwhere(diff_acc != 0)] = 1
  rise_edge = search_sequence_numpy(diff_acc, np.array([1,0]))
  fall_edge = search_sequence_numpy(diff_acc, np.array([0,1]))

  total = np.concatenate((rise_edge[::2], fall_edge[::2]), axis=None)
  total = np.sort(total)

  sum_enc = enc1+enc2
  diff_enc = np.diff(enc_sum)
  diff_enc[np.argwhere(diff_enc != 0)] = 1


  fig, axs = plt.subplots(nrows=5, ncols=1, figsize=(24,10))
  axs[0].plot(acc, label = 'ACC')
  axs[0].set_title("ACC")
  axs[0].set_xlim([0, 700])

  axs[1].plot(facc, label = 'facc')
  axs[1].set_title("filtered acc")
  axs[1].set_xlim([0, 700])

  axs[2].plot(diff_acc, label = 'diff_acc')
  axs[2].set_title("diff_acc")
  axs[2].set_xlim([0, 700])

  axs[3].plot(sum_enc, label = 'sum_enc')
  axs[3].set_title("sum_enc")
  axs[3].set_xlim([0, 700])

  axs[4].plot(diff_enc, label = 'diff_enc')
  axs[4].set_title("diff_enc")
  axs[4].set_xlim([0, 700])


  wheel = np.zeros(len(acc))
  air = np.zeros(len(acc))
  gnd = np.zeros(len(acc))

  for i in range(len(total) - 1):
    # where acc dot is almost zero and without bouncing
    if total[i+1] - total[i] >= win_size:
      # print((sum_enc[total[i+1]] - sum_enc[total[i]])/(total[i+1] - total[i]))
      if (sum_enc[total[i+1]] - sum_enc[total[i]]) != 0 and (sum_enc[total[i+1]] - sum_enc[total[i]])/(total[i+1] - total[i]) >= threshold: # the wheels are moving
        wheel[total[i] : total[i+1]] = 100 * np.ones(total[i+1] - total[i])
        for ax in axs:
          ax.axvspan(total[i], total[i+1], facecolor='green', alpha=0.2)
      else: # the wheels not moving, total stop
        continue
    # acc dot is not zero
    else:
      if (sum_enc[total[i+1]] - sum_enc[total[i]]) != 0: # the wheels are moving
        gnd[total[i] : total[i+1]] = 50 * np.ones(total[i+1] - total[i])
        for ax in axs:
          ax.axvspan(total[i], total[i+1], facecolor='red', alpha=0.2)
      else: # the wheels not moving, in the air
        air[total[i] : total[i+1]] = 75 * np.ones(total[i+1] - total[i])
        for ax in axs:
          ax.axvspan(total[i], total[i+1], facecolor='yellow', alpha=0.2)

  play_len = np.count_nonzero(wheel) + np.count_nonzero(air) + np.count_nonzero(gnd)

  table.append([name,
                np.count_nonzero(wheel)/play_len, np.count_nonzero(wheel)/len(acc),
                np.count_nonzero(air)/play_len, np.count_nonzero(air)/len(acc),
                np.count_nonzero(gnd)/play_len, np.count_nonzero(gnd)/len(acc),
                play_len/len(acc)])

  # for i in range(len(total) - 1):
  #   if total[i+1] - total[i] >= win_size:
  #     interval_size = total[i+1] - total[i]
  #     enc_diff = enc_sum[total[i+1]] - enc_sum[total[i]]
  #     ratio = enc_diff / interval_size
  #     rps = ratio * 4 

  #     print("interval_size:  ", interval_size, "  enc_diff:  ", enc_diff, '  ratio: ', ratio, "  rps:  ", rps)


  
  # fig = plt.figure(figsize=(24, 10))
  # grid = plt.GridSpec(2, 2)
  # acc_axs = fig.add_subplot(grid[0,0])
  # accdot_axs = fig.add_subplot(grid[0,1])
  # enc_axs = fig.add_subplot(grid[1,0])
  # encdot_axs = fig.add_subplot(grid[1,1])

  

  # x_ticks = np.arange(0, 700, 50)
  # axs[0].set_xticks(x_ticks)
  # axs[1].set_xticks(x_ticks)
  # axs[2].set_xticks(x_ticks)
  fig.suptitle(name)
  plt.show()

print(tabulate(table, headers=["name", "wheels (play time)", "wheels (total)",
                               "air (play time)", "air (total)",
                               "ground (play time)", "ground (total)",
                               "play time"],
               tablefmt='grid'))

In [ ]:
def single_enc(sum, win_size = 6):
  count = 0
  idxp = np.argwhere(sum == 1)
  idxn = np.argwhere(sum == -1)
  for indx in idxp:
    if not -1 in sum[indx[0] - win_size : indx[0] + win_size + 1]:
      # print(sum[indx[0] - win_size : indx[0] + win_size + 1])
      count = count + np.count_nonzero(sum[indx[0] - win_size : indx[0] + win_size + 1])
  for indx in idxn:
    if not 1 in sum[indx[0] - win_size : indx[0] + win_size + 1]:
      # print(sum[indx[0] - win_size : indx[0] + win_size + 1])
      count = count + np.count_nonzero(sum[indx[0] - win_size : indx[0] + win_size + 1])

  return count

In [ ]:
# event classification based on encoders

win_size = 8
threshold = 0.1 * win_size
table = []

for temp in filtered_data:
  id = temp[0]
  stat = temp[1]
  data = temp[2]

  name = str(id)+stat

  time = data[0]

  accx = data[1]
  accy = data[2]
  accz = data[3]

  enc1 = data[4]
  enc2 = data[5]

  diff_enc1 = np.diff(enc1)
  diff_enc1[np.argwhere(diff_enc1 != 0)] = 1

  diff_enc2 = np.diff(enc2)
  diff_enc2[np.argwhere(diff_enc2 != 0)] = -1

  diff_enc1 = diff_enc1 +1
  diff_enc2 = diff_enc2 -1


  facxdot = zero_filter(np.diff(accx), 0.05)
  facydot = zero_filter(np.diff(accy), 0.05)
  faczdot = zero_filter(np.diff(accz), 0.05)



  # for i in range(len(min(enc1,enc2) - win_size)):
  #   if enc1[i + win_size] - enc1[i] >= threshold and enc2[i + win_size] - enc2[i] < threshold:



  # fig, axs = plt.subplots(nrows=3, ncols=2, figsize=(24,10))
  # fig = plt.figure(figsize=(24, 10))
  # grid = plt.GridSpec(3, 2)
  # enc_plot = fig.add_subplot(grid[0,:])
  # enc1dot_plot = fig.add_subplot(grid[1,0])
  # enc2dot_plot = fig.add_subplot(grid[1,1])
  # sum_plot = fig.add_subplot(grid[2,:])

  # enc_plot.plot(enc1, label = 'enc1')
  # enc_plot.plot(enc2, label = 'enc2')
  # enc_plot.set_title("enc")
  # enc_plot.legend()
  # enc_plot.set_xlim([0, 700])

  # enc1dot_plot.plot(diff_enc1, label = 'diff_enc1')
  # enc1dot_plot.set_title("diff_enc1")
  # enc1dot_plot.set_xlim([0, 700])

  # enc2dot_plot.plot(diff_enc2, label = 'diff_enc2')
  # enc2dot_plot.set_title("diff_enc2")
  # enc2dot_plot.set_xlim([0, 700])

  # sum_plot.plot(diff_enc1, label = 'diff_enc1')
  # sum_plot.plot(diff_enc2, label = 'diff_enc2')
  # sum_plot.plot(diff_enc1 + diff_enc2, label = 'diff_enc_sum')
  # sum_plot.set_title('encdot_sum')
  # sum_plot.legend()
  # sum_plot.set_xlim([0, 700])


  

  # diff_acxdot = np.copy(facxdot)
  # diff_acydot = np.copy(facydot)
  # diff_aczdot = np.copy(faczdot)

  # diff_acc[np.argwhere(diff_acc != 0)] = 1
  # rise_edge = search_sequence_numpy(diff_acc, np.array([1,0]))
  # fall_edge = search_sequence_numpy(diff_acc, np.array([0,1]))

  # total = np.concatenate((rise_edge[::2], fall_edge[::2]), axis=None)
  # total = np.sort(total)

  # sum_enc = enc1+enc2
  # diff_enc = np.diff(enc_sum)
  # diff_enc[np.argwhere(diff_enc != 0)] = 1

  sum = diff_enc1 + diff_enc2
  print(name, " ", single_enc(sum,16))

  # fig.suptitle(name)
  # plt.show()

In [ ]:
# plot filltered data varance and mean

interval_window = 16
table = []

for temp in filtered_data:
  id = temp[0]
  stat = temp[1]
  data = temp[2]

  name = str(id)+stat

  time = data[0]

  accx = data[1]
  accy = data[2]
  accz = data[3]

  enc1 = data[4]
  enc2 = data[5]

  fig, axs = plt.subplots(nrows=5, ncols=1, figsize=(24,16))
  axs[0].plot(accx, label = 'ACx')
  axs[0].plot(np.diff(accx), label = 'ACx dot')
  # axs[0].axhline(y=np.mean(accx), color='r', linestyle=':', linewidth= '3', label = 'mean Acx')
  # axs[0].axhline(y=np.std(accx), color='g', linestyle=':', linewidth= '3', label = 'std Acx')
  # axs[0].axhline(y=-1 * np.std(accx), color='g', linestyle=':', linewidth= '3', label = '- std Acx')
  axs[0].legend()
  axs[0].set_title("Acx")

  axs[1].plot(accy, label = 'ACy')
  axs[1].axhline(y=np.mean(accy), color='r', linestyle=':', linewidth= '3', label = 'mean Acy')
  axs[1].axhline(y=np.std(accy), color='g', linestyle=':', linewidth= '3', label = 'std Acy')
  axs[1].axhline(y=-1 * np.std(accy), color='g', linestyle=':', linewidth= '3', label = '- std Acy')
  axs[1].legend()
  axs[1].set_title("Acy")

  axs[2].plot(accz, label = 'ACz')
  axs[2].axhline(y=np.mean(accz), color='r', linestyle=':', linewidth= '3', label = 'mean Acz')
  axs[2].axhline(y=np.std(accz), color='g', linestyle=':', linewidth= '3', label = 'std Acz')
  axs[2].axhline(y=-1 * np.std(accz), color='g', linestyle=':', linewidth= '3', label = '- std Acz')
  axs[2].legend()
  axs[2].set_title("Acz")

  axs[3].plot(enc1, label = 'enc1')
  axs[3].set_title("enc1")

  axs[4].plot(enc2, label = 'enc2')
  axs[4].set_title("enc2")

  fig.suptitle(name)
  plt.show() 
  

In [ ]:
def mod_diff(input_array, interval_size = 2):
  diff_mat = np.zeros(len(input_array))
  for i in range(len(input_array) - interval_size):
    diff_mat[i : i + interval_size] = (input_array[i + interval_size] - input_array[i]) * np.ones(interval_size)

  return diff_mat

In [ ]:
def group_consecutives(vals, step=1):
    """Return list of consecutive lists of numbers from vals (number list)."""
    run = []
    result = [run]
    expect = None
    for v in vals:
        if (v == expect) or (expect is None):
            run.append(v)
        else:
            run = [v]
            result.append(run)
        expect = v + step
    return result

In [ ]:
win_size = 10
enc_threshold = 3
accdot_filter_threshold = 0.02
table = []

for temp in filtered_data:
  id = temp[0]
  stat = temp[1]
  data = temp[2]

  name = str(id)+stat

  time = data[0]

  accx = data[1]
  accy = data[2]
  accz = data[3]

  enc1 = data[4]
  enc2 = data[5]

  acc = np.sqrt(accx **2 + accy **2 + accz **2)
  facc = mod_zero_filter(np.diff(acc), accdot_filter_threshold)
  diff_acc = np.copy(facc)

  stop_rng = group_consecutives(np.where(diff_acc == 0)[0], 1)

  sum_enc = enc1+enc2
  diff_enc = np.diff(enc_sum)
  diff_enc[np.argwhere(diff_enc != 0)] = 1


  fig, axs = plt.subplots(nrows=3, ncols=1, figsize=(24,8))
  axs[0].plot(acc, label = 'ACC')
  axs[0].set_title("ACC")
  axs[0].set_xlim([0, 700])

  axs[1].plot(diff_acc, label = 'diff_acc')
  axs[1].set_title("diff_acc")
  axs[1].set_xlim([0, 700])

  axs[2].plot(sum_enc, label = 'sum_enc')
  axs[2].set_title("sum_enc")
  axs[2].set_xlim([0, 700])


  wheel = 0
  not_play = 0
  other = 0

  for rng in stop_rng:
    if len(rng) >= win_size:
      if sum_enc[rng[-1]] - sum_enc[rng[0]] >= enc_threshold:
        # wheel only
        wheel = wheel + len(rng)
        for ax in axs:
          ax.axvspan(rng[-1], rng[0], facecolor='green', alpha=0.4)
      else:
        # not playing
        not_play = not_play + len(rng)
        for ax in axs:
          ax.axvspan(rng[-1], rng[0], facecolor='red', alpha=0.4)
    else:
      other = other + len(rng)
      for ax in axs:
          ax.axvspan(rng[-1], rng[0], facecolor='yellow', alpha=0.4) 


  play = len(acc) - wheel - not_play - other
  table.append([name,
                wheel, wheel/len(acc),
                not_play, not_play/len(acc),
                play, play/len(acc),
                other, other/len(acc),
                wheel/play])

  fig.suptitle(name)
  plt.show()

print(tabulate(table, headers=["name",
                               "wheels", "wheels ratio",
                               "not_play", "not_play ratio",
                               "play", "play ratio",
                               "other", "other ratio",
                               "wheel ratio (play time)"],
               tablefmt='grid'))

In [ ]:
def window_filter(input_array, window_size):
  if window_size % 2 == 0:
    return -1
  out = []
  for i in range(len(input_array)):
    if i%window_size == 0 or i == len(input_array):
      out[i - window_size : i] = np.ones(window_size) if np.count_nonzero(input_array[i - window_size : i]) >= window_size - 2 else np.zeros(window_size)
  return out

In [ ]:
def state_cntr(accdot, encdot):
  stop = 0
  wheel = 0
  air = 0
  gnd = 0
  l = min(len(accdot), len(encdot))
  for i in range(l):
    if accdot[i] == 0 and encdot[i] == 0:
      stop += 1
    elif accdot[i] == 0 and encdot[i] == 1:
      wheel += 1
    elif accdot[i] == 1 and encdot[i] == 0:
      air += 1
    elif accdot[i] == 1 and encdot[i] == 1:
      gnd += 1
    else:
      print("ERROR")
      break

  output = [stop/l, wheel/l, air/l, gnd/l]
  return output

In [ ]:
def just_wheel(accdot, encdot):
  output = []
  l = min(len(accdot), len(encdot))
  for i in range(l):
    if accdot[i] == 0 and encdot[i] == 1:
      output.append(1)
    else:
      output.append(0)
  output = np.array(output)
  return output

In [ ]:
table = []

for temp in filtered_data:
  id = temp[0]
  stat = temp[1]
  data = temp[2]

  acx = data[1]
  acy = data[2]
  acz = data[3]

  acx_mean = np.mean(acx)
  acy_mean = np.mean(acy)
  acz_mean = np.mean(acz)

  acx_var = np.std(acx)
  acy_var = np.std(acy)
  acz_var = np.std(acz)

  table.append([id,stat,acx_mean,acy_mean,acz_mean,acx_var,acy_var,acz_var])

print(tabulate(table, headers=["id", "stat", "acx_mean", "acy_mean", "acz_mean","acx_var", "acy_var", "acz_var"], tablefmt='grid'))  

In [ ]:
# mean btw all signals and multiply and fft each one with them
# Not compelete

n_acc = []
a_acc = []
na_acc = []

for data in filtered_data:
  name = str(data[0])+data[1]
  # print(name)
  accx = data[2][1]
  accy = data[2][2]
  accz = data[2][3]

  acc_data = [accx, accy, accz]

  if name.find("normal") > 1:
    n_acc.append(acc_data)
  elif name.find("autistic") > 1:
    a_acc.append(acc_data)
  elif name.find("NA") > 1:
    na_acc.append(acc_data)
  else:
    print("error")

n_accx_mean = np.mean(n_acc[0], axis = 0)
n_accx_mean = np.pad(n_accx_mean, (700 - n_accx_mean.shape[0] - 1, 1), 'constant', constant_values=(1,1))
n_accy_mean = np.mean(n_acc[1], axis = 0)
n_accy_mean = np.pad(n_accy_mean, (700 - n_accy_mean.shape[0] - 1, 1), 'constant', constant_values=(1,1))
n_accz_mean = np.mean(n_acc[2], axis = 0)
n_accz_mean = np.pad(n_accz_mean, (700 - n_accz_mean.shape[0] - 1, 1), 'constant', constant_values=(1,1))

a_accx_mean = np.mean(a_acc[0], axis = 0)
a_accx_mean = np.pad(a_accx_mean, (700 - a_accx_mean.shape[0] - 1, 1), 'constant', constant_values=(1,1))
a_accy_mean = np.mean(a_acc[1], axis = 0)
a_accy_mean = np.pad(a_accy_mean, (700 - a_accy_mean.shape[0] - 1, 1), 'constant', constant_values=(1,1))
a_accz_mean = np.mean(a_acc[2], axis = 0)
a_accz_mean = np.pad(a_accz_mean, (700 - a_accz_mean.shape[0] - 1, 1), 'constant', constant_values=(1,1))

na_accx_mean = np.mean(na_acc[0], axis = 0)
na_accx_mean = np.pad(na_accx_mean, (700 - na_accx_mean.shape[0] - 1, 1), 'constant', constant_values=(1,1))
na_accy_mean = np.mean(na_acc[1], axis = 0)
na_accy_mean = np.pad(na_accy_mean, (700 - na_accy_mean.shape[0] - 1, 1), 'constant', constant_values=(1,1))
na_accz_mean = np.mean(na_acc[2], axis = 0)
na_accz_mean = np.pad(na_accz_mean, (700 - na_accz_mean.shape[0] - 1, 1), 'constant', constant_values=(1,1))

for data in filtered_data:
  name = str(data[0])+data[1]
  # print(name)
  accx = np.pad(data[2][1], (700 - data[2][1].shape[0] - 1, 1), 'constant', constant_values=(1,1))
  accy = np.pad(data[2][2], (700 - data[2][2].shape[0] - 1, 1), 'constant', constant_values=(1,1))
  accz = np.pad(data[2][3], (700 - data[2][3].shape[0] - 1, 1), 'constant', constant_values=(1,1))

  if name.find("normal") > 1:
    print(name)
    x_product = n_accx_mean * accx
    y_product = n_accy_mean * accy
    z_product = n_accz_mean * accz

    fig, axs = plt.subplots(nrows=3, ncols=3, figsize=(20, 8))

    axs[0,0].plot(accx)
    axs[0,0].set_title("ACx")
    axs[0,1].plot(x_product)
    axs[0,1].set_title("x_product")
    axs[0,2].plot(np.fft.fft(x_product))
    axs[0,2].set_title("fft")

    axs[1,0].plot(accy)
    axs[1,0].set_title("ACy")
    axs[1,1].plot(y_product)
    axs[1,1].set_title("y_product")
    axs[1,2].plot(np.fft.fft(y_product))
    axs[1,2].set_title("fft")

    axs[2,0].plot(accz)
    axs[2,0].set_title("ACz")
    axs[2,1].plot(z_product)
    axs[2,1].set_title("z_product")
    axs[2,2].plot(np.fft.fft(z_product))
    axs[2,2].set_title("fft")

    fig.suptitle(name)
    plt.show()

    print("------------------------------------------------------------------")
    
  elif name.find("autistic") > 1:
    print(name)
    x_product = a_accx_mean * accx
    y_product = a_accy_mean * accy
    z_product = a_accz_mean * accz


    fig, axs = plt.subplots(nrows=3, ncols=3, figsize=(20, 8))

    axs[0,0].plot(accx)
    axs[0,0].set_title("ACx")
    axs[0,1].plot(x_product)
    axs[0,1].set_title("x_product")
    axs[0,2].plot(np.fft.fft(x_product))
    axs[0,2].set_title("fft")

    axs[1,0].plot(accy)
    axs[1,0].set_title("ACy")
    axs[1,1].plot(y_product)
    axs[1,1].set_title("y_product")
    axs[1,2].plot(np.fft.fft(y_product))
    axs[1,2].set_title("fft")

    axs[2,0].plot(accz)
    axs[2,0].set_title("ACz")
    axs[2,1].plot(z_product)
    axs[2,1].set_title("z_product")
    axs[2,2].plot(np.fft.fft(z_product))
    axs[2,2].set_title("fft")

    fig.suptitle(name)
    plt.show()

    print("------------------------------------------------------------------")

  elif name.find("NA") > 1:
    print(name)
    x_product = na_accx_mean * accx
    y_product = na_accy_mean * accy
    z_product = na_accz_mean * accz

    fig, axs = plt.subplots(nrows=3, ncols=3, figsize=(20, 8))

    axs[0,0].plot(accx)
    axs[0,0].set_title("ACx")
    axs[0,1].plot(x_product)
    axs[0,1].set_title("x_product")
    axs[0,2].plot(np.fft.fft(x_product))
    axs[0,2].set_title("fft")

    axs[1,0].plot(accy)
    axs[1,0].set_title("ACy")
    axs[1,1].plot(y_product)
    axs[1,1].set_title("y_product")
    axs[1,2].plot(np.fft.fft(y_product))
    axs[1,2].set_title("fft")

    axs[2,0].plot(accz)
    axs[2,0].set_title("ACz")
    axs[2,1].plot(z_product)
    axs[2,1].set_title("z_product")
    axs[2,2].plot(np.fft.fft(z_product))
    axs[2,2].set_title("fft")

    fig.suptitle(name)
    plt.show()

    print("------------------------------------------------------------------")
    
  else:
    print("error")

In [ ]:
from scipy import signal
from matplotlib import cm

specto_list = []

for data in filtered_data:
  name = str(data[0])+data[1]
  # print(name)
  accx = data[2][1]
  accy = data[2][2]
  accz = data[2][3]

  rate= 1
  Block_size=8

  fig, (axx, axy, axz) = plt.subplots(nrows=1, ncols=3, figsize=(24, 8))

  freqs, times, Sxx = signal.spectrogram(accx,
                                         fs=rate,
                                         noverlap=0.5*Block_size,
                                         detrend = "linear",
                                         nperseg=Block_size,
                                         scaling='spectrum')
  
  pcmx = axx.pcolormesh(times, freqs, Sxx, cmap='inferno')
  fig.colorbar(pcmx, ax = axx)
  
  freqs, times, Syy = signal.spectrogram(accy,
                                         fs=rate,
                                         noverlap=0.5*Block_size,
                                         detrend = "linear",
                                         nperseg=Block_size,
                                         scaling='spectrum')

  pcmy = axy.pcolormesh(times, freqs, Syy, cmap='inferno')
  fig.colorbar(pcmy, ax = axy)

  freqs, times, Szz = signal.spectrogram(accz,
                                         fs=rate,
                                         noverlap=0.5*Block_size,
                                         detrend = "linear",
                                         nperseg=Block_size,
                                         scaling='spectrum')

  pcmz = axz.pcolormesh(times, freqs, Szz, cmap='inferno')
  fig.colorbar(pcmz, ax = axz)

  tempx = np.zeros((5,180))
  tempx[0:Sxx.shape[0], 0: Sxx.shape[1]] = Sxx
  
  tempy = np.zeros((5,180))
  tempy[0:Syy.shape[0], 0: Syy.shape[1]] = Syy

  tempz = np.zeros((5,180))
  tempz[0:Szz.shape[0], 0: Szz.shape[1]] = Szz

  specto_list.append([name, tempx, tempy, tempz])

  
  fig.suptitle(name)
  axx.set_title("X")
  axx.set(xlabel='Time [s]', ylabel='Frequency [Hz]')
  axy.set_title("Y")
  axy.set(xlabel='Time [s]', ylabel='Frequency [Hz]')
  axz.set_title("Z")
  axz.set(xlabel='Time [s]', ylabel='Frequency [Hz]')

  plt.show()


In [ ]:
n_specto = []
a_specto = []
na_specto = []

for specto in specto_list:

  name = specto[0]
  spec_x = specto[1]
  spec_y = specto[2]
  spec_z = specto[3]

  spec_data = [spec_x, spec_y, spec_z]

  if name.find("normal") > 1:
    n_specto.append(spec_data)
  elif name.find("autistic") > 1:
    a_specto.append(spec_data)
  elif name.find("NA") > 1:
    na_specto.append(spec_data)
  else:
    print("error")

  # print(name)
  # fig, ax = plt.subplots()
  # im = ax.imshow(spec_data, origin = 'lower', cmap=plt.get_cmap('inferno'), interpolation='none', aspect='auto')
  # fig.colorbar(im)
  # plt.show()

In [ ]:
# another comp on specto
# cross correlation btw means and each signals

n_x_specto_mean = np.mean(n_specto[0], axis = 0)
n_y_specto_mean = np.mean(n_specto[1], axis = 0)
n_z_specto_mean = np.mean(n_specto[2], axis = 0)

a_x_specto_mean = np.mean(a_specto[0], axis = 0)
a_y_specto_mean = np.mean(a_specto[1], axis = 0)
a_z_specto_mean = np.mean(a_specto[2], axis = 0)

na_x_specto_mean = np.mean(na_specto[0], axis = 0)
na_y_specto_mean = np.mean(na_specto[1], axis = 0)
na_z_specto_mean = np.mean(na_specto[2], axis = 0)

for specto in specto_list:

  name = specto[0]
  spec_x = specto[1]
  spec_y = specto[2]
  spec_z = specto[3]

  corr_n_x = signal.correlate(n_x_specto_mean, spec_x, method='direct')
  corr_n_y = signal.correlate(n_y_specto_mean, spec_y, method='direct')
  corr_n_z = signal.correlate(n_z_specto_mean, spec_z, method='direct')

  corr_a_x = signal.correlate(a_x_specto_mean, spec_x, method='direct')
  corr_a_y = signal.correlate(a_y_specto_mean, spec_y, method='direct')
  corr_a_z = signal.correlate(a_z_specto_mean, spec_z, method='direct')

  corr_na_x = signal.correlate(na_x_specto_mean, spec_x, method='direct')
  corr_na_y = signal.correlate(na_y_specto_mean, spec_y, method='direct')
  corr_na_z = signal.correlate(na_z_specto_mean, spec_z, method='direct')

  fig, axs = plt.subplots(nrows=3, ncols=3, figsize=(20,20))

  nx = axs[0,0].imshow(corr_n_x, origin = 'lower', cmap=plt.get_cmap('inferno'), interpolation='none', aspect='auto')
  axs[0,0].set_title("corr_n_x")
  fig.colorbar(nx, ax=axs[0,0])
  ny = axs[0,1].imshow(corr_n_y, origin = 'lower', cmap=plt.get_cmap('inferno'), interpolation='none', aspect='auto')
  axs[0,1].set_title("corr_n_y")
  fig.colorbar(ny, ax=axs[0,1])
  nz = axs[0,2].imshow(corr_n_z, origin = 'lower', cmap=plt.get_cmap('inferno'), interpolation='none', aspect='auto')
  axs[0,2].set_title("corr_n_z")
  fig.colorbar(nz, ax=axs[0,2])


  ax = axs[1,0].imshow(corr_a_x, origin = 'lower', cmap=plt.get_cmap('inferno'), interpolation='none', aspect='auto')
  axs[1,0].set_title("corr_a_x")
  fig.colorbar(ax, ax=axs[1,0])
  ay = axs[1,1].imshow(corr_a_y, origin = 'lower', cmap=plt.get_cmap('inferno'), interpolation='none', aspect='auto')
  axs[1,1].set_title("corr_a_y")
  fig.colorbar(ay, ax=axs[1,1])
  az = axs[1,2].imshow(corr_a_z, origin = 'lower', cmap=plt.get_cmap('inferno'), interpolation='none', aspect='auto')
  axs[1,2].set_title("corr_a_z")
  fig.colorbar(az, ax=axs[1,2])


  nax = axs[2,0].imshow(corr_na_x, origin = 'lower', cmap=plt.get_cmap('inferno'), interpolation='none', aspect='auto')
  axs[2,0].set_title("corr_na_x")
  fig.colorbar(nax, ax=axs[2,0])
  nay = axs[2,1].imshow(corr_na_y, origin = 'lower', cmap=plt.get_cmap('inferno'), interpolation='none', aspect='auto')
  axs[2,1].set_title("corr_na_y")
  fig.colorbar(nay, ax=axs[2,1])
  naz = axs[2,2].imshow(corr_na_z, origin = 'lower', cmap=plt.get_cmap('inferno'), interpolation='none', aspect='auto')
  axs[2,2].set_title("corr_na_z")
  fig.colorbar(naz, ax=axs[2,2])

  fig.suptitle(name)
  plt.show()

In [ ]:
# another comp on specto
# absolute diffrenece btw means and specto

n_x_specto_mean = np.mean(n_specto[0], axis = 0)
n_y_specto_mean = np.mean(n_specto[1], axis = 0)
n_z_specto_mean = np.mean(n_specto[2], axis = 0)

a_x_specto_mean = np.mean(a_specto[0], axis = 0)
a_y_specto_mean = np.mean(a_specto[1], axis = 0)
a_z_specto_mean = np.mean(a_specto[2], axis = 0)

na_x_specto_mean = np.mean(na_specto[0], axis = 0)
na_y_specto_mean = np.mean(na_specto[1], axis = 0)
na_z_specto_mean = np.mean(na_specto[2], axis = 0)

for specto in specto_list:

  name = specto[0]
  spec_x = specto[1]
  spec_y = specto[2]
  spec_z = specto[3]

  diff_nx = abs(n_x_specto_mean - spec_x)
  diff_ny = abs(n_y_specto_mean - spec_y)
  diff_nz = abs(n_z_specto_mean - spec_z)

  diff_ax = abs(a_x_specto_mean - spec_x)
  diff_ay = abs(a_y_specto_mean - spec_y)
  diff_az = abs(a_z_specto_mean - spec_z)

  diff_nax = abs(na_x_specto_mean - spec_x)
  diff_nay = abs(na_y_specto_mean - spec_y)
  diff_naz = abs(na_z_specto_mean - spec_z)

  fig, axs = plt.subplots(nrows=3, ncols=3, figsize=(20,20))

  nx = axs[0,0].imshow(diff_nx, origin = 'lower', cmap=plt.get_cmap('inferno'), interpolation='none', aspect='auto')
  axs[0,0].set_title("diff nx")
  fig.colorbar(nx, ax=axs[0,0])
  ny = axs[0,1].imshow(diff_ny, origin = 'lower', cmap=plt.get_cmap('inferno'), interpolation='none', aspect='auto')
  axs[0,1].set_title("diff ny")
  fig.colorbar(ny, ax=axs[0,1])
  nz = axs[0,2].imshow(diff_nz, origin = 'lower', cmap=plt.get_cmap('inferno'), interpolation='none', aspect='auto')
  axs[0,2].set_title("diff nz")
  fig.colorbar(nz, ax=axs[0,2])


  ax = axs[1,0].imshow(diff_ax, origin = 'lower', cmap=plt.get_cmap('inferno'), interpolation='none', aspect='auto')
  axs[1,0].set_title("diff ax")
  fig.colorbar(ax, ax=axs[1,0])
  ay = axs[1,1].imshow(diff_ay, origin = 'lower', cmap=plt.get_cmap('inferno'), interpolation='none', aspect='auto')
  axs[1,1].set_title("diff ay")
  fig.colorbar(ay, ax=axs[1,1])
  az = axs[1,2].imshow(diff_az, origin = 'lower', cmap=plt.get_cmap('inferno'), interpolation='none', aspect='auto')
  axs[1,2].set_title("diff az")
  fig.colorbar(az, ax=axs[1,2])


  nax = axs[2,0].imshow(diff_nax, origin = 'lower', cmap=plt.get_cmap('inferno'), interpolation='none', aspect='auto')
  axs[2,0].set_title("diff nax")
  fig.colorbar(nax, ax=axs[2,0])
  nay = axs[2,1].imshow(diff_nay, origin = 'lower', cmap=plt.get_cmap('inferno'), interpolation='none', aspect='auto')
  axs[2,1].set_title("diff nay")
  fig.colorbar(nay, ax=axs[2,1])
  naz = axs[2,2].imshow(diff_naz, origin = 'lower', cmap=plt.get_cmap('inferno'), interpolation='none', aspect='auto')
  axs[2,2].set_title("diff naz")
  fig.colorbar(naz, ax=axs[2,2])

  fig.suptitle(name)
  plt.show()

In [ ]:
#  distanc calc reshaped arrayes 

from scipy.spatial import distance

n_x_specto_mean = np.reshape(np.mean(n_specto[0], axis = 0), (1,900))
n_y_specto_mean = np.reshape(np.mean(n_specto[1], axis = 0), (1,900))
n_z_specto_mean = np.reshape(np.mean(n_specto[2], axis = 0), (1,900))

a_x_specto_mean = np.reshape(np.mean(a_specto[0], axis = 0), (1,900))
a_y_specto_mean = np.reshape(np.mean(a_specto[1], axis = 0), (1,900))
a_z_specto_mean = np.reshape(np.mean(a_specto[2], axis = 0), (1,900))

na_x_specto_mean = np.reshape(np.mean(na_specto[0], axis = 0), (1,900))
na_y_specto_mean = np.reshape(np.mean(na_specto[1], axis = 0), (1,900))
na_z_specto_mean = np.reshape(np.mean(na_specto[2], axis = 0), (1,900))

table = []
mean_table = []
for specto in specto_list:

  name = specto[0]
  spec_x = np.reshape(specto[1], (1,900))
  spec_y = np.reshape(specto[2], (1,900))
  spec_z = np.reshape(specto[3], (1,900))

  dis_n_x = 1 - distance.cosine(n_x_specto_mean, spec_x)
  dis_n_y = 1 - distance.cosine(n_y_specto_mean, spec_y)
  dis_n_z = 1 - distance.cosine(n_z_specto_mean, spec_z)

  dis_a_x = 1 - distance.cosine(a_x_specto_mean, spec_x)
  dis_a_y = 1 - distance.cosine(a_y_specto_mean, spec_y)
  dis_a_z = 1 - distance.cosine(a_z_specto_mean, spec_z)

  dis_na_x = 1 - distance.cosine(na_x_specto_mean, spec_x)
  dis_na_y = 1 - distance.cosine(na_y_specto_mean, spec_y)
  dis_na_z = 1 - distance.cosine(na_z_specto_mean, spec_z)

  mean_n = np.mean([dis_n_x, dis_n_y, dis_n_z])
  mean_a = np.mean([dis_a_x, dis_a_y, dis_a_z])
  mean_na = np.mean([dis_na_x, dis_na_y, dis_na_z])

  mean_table.append([name, mean_n, mean_a, mean_na])
  table.append([name, dis_n_x, dis_n_y, dis_n_z, dis_a_x, dis_a_y, dis_a_z, dis_na_x, dis_na_y, dis_na_z])
print(tabulate(mean_table, headers=["name", "dis (N)", "dis (A)", "dis (NA)"], tablefmt='grid'))
print(tabulate(table, headers=["name", "dis X (N)", "dis Y (N)", "dis Z (N)", "dis X (A)", "dis Y (A)", "dis Z (A)", "dis X (NA)", "dis Y (NA)", "dis Z (NA)"], tablefmt='grid'))

In [ ]:
# distance calc btw means

from scipy.spatial import distance

n_x_specto_mean = np.mean(np.mean(n_specto[0], axis = 0), axis = 0)
n_y_specto_mean = np.mean(np.mean(n_specto[1], axis = 0), axis = 0)
n_z_specto_mean = np.mean(np.mean(n_specto[2], axis = 0), axis = 0)

a_x_specto_mean = np.mean(np.mean(a_specto[0], axis = 0), axis = 0)
a_y_specto_mean = np.mean(np.mean(a_specto[1], axis = 0), axis = 0)
a_z_specto_mean = np.mean(np.mean(a_specto[2], axis = 0), axis = 0)

na_x_specto_mean = np.mean(np.mean(na_specto[0], axis = 0), axis = 0)
na_y_specto_mean = np.mean(np.mean(na_specto[1], axis = 0), axis = 0)
na_z_specto_mean = np.mean(np.mean(na_specto[2], axis = 0), axis = 0)

table = []
mean_table = []
for specto in specto_list:

  name = specto[0]
  spec_x = specto[1]
  spec_y = specto[2]
  spec_z = specto[3]

  dis_n_x = 1 - distance.cosine(n_x_specto_mean, np.mean(spec_x, axis=0))
  dis_n_y = 1 - distance.cosine(n_y_specto_mean, np.mean(spec_y, axis=0))
  dis_n_z = 1 - distance.cosine(n_z_specto_mean, np.mean(spec_z, axis=0))

  dis_a_x = 1 - distance.cosine(a_x_specto_mean, np.mean(spec_x, axis=0))
  dis_a_y = 1 - distance.cosine(a_y_specto_mean, np.mean(spec_y, axis=0))
  dis_a_z = 1 - distance.cosine(a_z_specto_mean, np.mean(spec_z, axis=0))

  dis_na_x = 1 - distance.cosine(na_x_specto_mean, np.mean(spec_x, axis=0))
  dis_na_y = 1 - distance.cosine(na_y_specto_mean, np.mean(spec_y, axis=0))
  dis_na_z = 1 - distance.cosine(na_z_specto_mean, np.mean(spec_z, axis=0))

  mean_n = np.mean([dis_n_x, dis_n_y, dis_n_z])
  mean_a = np.mean([dis_a_x, dis_a_y, dis_a_z])
  mean_na = np.mean([dis_na_x, dis_na_y, dis_na_z])

  mean_table.append([name, mean_n, mean_a, mean_na])
  table.append([name, dis_n_x, dis_n_y, dis_n_z, dis_a_x, dis_a_y, dis_a_z, dis_na_x, dis_na_y, dis_na_z])
print(tabulate(mean_table, headers=["name", "dis (N)", "dis (A)", "dis (NA)"], tablefmt='grid'))
print(tabulate(table, headers=["name", "dis X (N)", "dis Y (N)", "dis Z (N)", "dis X (A)", "dis Y (A)", "dis Z (A)", "dis X (NA)", "dis Y (NA)", "dis Z (NA)"], tablefmt='grid'))

In [ ]:
# save and load data as numpy file


n_x_spec = []
a_x_spec = []
na_x_spec = []

for specto in specto_list:

  name = specto[0]
  spec_x = specto[1]
  spec_y = specto[2]
  spec_z = specto[3]

  save("%s.npy" %name, spec_x)

  if name.find("normal") > 1:
    n_x_spec.append(spec_x)
  elif name.find("autistic") > 1:
    a_x_spec.append(spec_x)
  elif name.find("NA") > 1:
    na_x_spec.append(spec_x)
  else:
    print("error")

_mean = np.load("/content/x_mean.npy")